In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.driver.memory", "10g").appName("Parquet Reader").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/19 12:34:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/19 12:34:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# Initialize the airline variable globally
airline = None

def read_parquet_and_print_top(parquet_path):
    global airline
    # Read Parquet files into a DataFrame
    airline = spark.read.parquet(parquet_path)

# Initialize Spark session and read Parquet file
parquet_path = "./airline.parquet"
read_parquet_and_print_top(parquet_path)

In [3]:
# get cols 
airline.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: date (nullable = true)
 |-- Reporting_Airline: string (nullable = true)
 |-- DOT_ID_Reporting_Airline: integer (nullable = true)
 |-- IATA_CODE_Reporting_Airline: string (nullable = true)
 |-- Tail_Number: string (nullable = true)
 |-- Flight_Number_Reporting_Airline: integer (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- OriginAirportSeqID: integer (nullable = true)
 |-- OriginCityMarketID: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCityName: string (nullable = true)
 |-- OriginState: string (nullable = true)
 |-- OriginStateFips: integer (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- OriginWac: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DestAirportSeqID: 

In [ ]:
flights_month = {}

months = airline.select('Month').distinct().collect()
months = [row.Month for row in months]
# assign total and delayed for each month
for month in months:
    total = airline.filter(airline.Month == month).count()
    delayed = airline.filter(airline.Month == month).filter(airline.ArrDelay > 0).count()
    flights_month[month] = {'total': total, 'delayed': delayed}
print(flights_month)

In [3]:
airline.columns

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Reporting_Airline',
 'DOT_ID_Reporting_Airline',
 'IATA_CODE_Reporting_Airline',
 'Tail_Number',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Flights',
 'Distance',
 'DistanceGroup',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityD

In [17]:
# records where delay is more than 15
delayed_flights = airline.filter((airline.ArrDelay > 15) & (airline.Distance > 2000)).count()
delayed_flights

1870334

In [4]:
airline.head(5)

24/03/19 12:34:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(Year=2019, Quarter=2, Month=4, DayofMonth=30, DayOfWeek=2, FlightDate=datetime.date(2019, 4, 30), Reporting_Airline='AA', DOT_ID_Reporting_Airline=19805, IATA_CODE_Reporting_Airline='AA', Tail_Number='N733UW', Flight_Number_Reporting_Airline=2119, OriginAirportID=10721, OriginAirportSeqID=1072102, OriginCityMarketID=30721, Origin='BOS', OriginCityName='Boston, MA', OriginState='MA', OriginStateFips=25, OriginStateName='Massachusetts', OriginWac=13, DestAirportID=11278, DestAirportSeqID=1127805, DestCityMarketID=30852, Dest='DCA', DestCityName='Washington, DC', DestState='VA', DestStateFips=51, DestStateName='Virginia', DestWac=38, CRSDepTime=1600, DepTime=1556, DepDelay=-4.0, DepDelayMinutes=0.0, DepDel15=0.0, DepartureDelayGroups=-1, DepTimeBlk='1600-1659', TaxiOut=15.0, WheelsOff='1611', WheelsOn='1723', TaxiIn=3.0, CRSArrTime=1745, ArrTime=1726, ArrDelay=-19.0, ArrDelayMinutes=0.0, ArrDel15=0.0, ArrivalDelayGroups=-2, ArrTimeBlk='1700-1759', Cancelled=0.0, CancellationCode=None

In [12]:
def get_top_airports(airline, years):
    top_airports = []
    result = {}
    for year in years:
        # Filter flights by the specified year and on-time departures
        on_time_flights = airline.filter(
            (airline['Year'] == year) & (airline['DepDelay'] <= 0))

        # Group flights by originating airport and count occurrences also get the OriginStateName and OriginCityName
        airport_counts = on_time_flights.groupBy( 'Origin', 'OriginStateName', 'OriginCityName').count() 
        top_airports_year = airport_counts.orderBy(
            airport_counts['count'].desc()).limit(3)

        # get the OriginStateName and OriginCityName
        top_airports = []
        for i, airport in enumerate(top_airports_year.collect(), start=1):
            top_airports.append(
                {"airport": airport['Origin'], "count": airport['count'] ,"OriginStateName": airport['OriginStateName'], "OriginCityName": airport['OriginCityName']})
        result[year] = top_airports
    return result

years = [1987, 1997, 2007, 2017]
result = get_top_airports(airline,years)
result

{1987: [{'airport': 'ATL',
   'count': 42765,
   'OriginStateName': 'Georgia',
   'OriginCityName': 'Atlanta, GA'},
  {'airport': 'ORD',
   'count': 28310,
   'OriginStateName': 'Illinois',
   'OriginCityName': 'Chicago, IL'},
  {'airport': 'DEN',
   'count': 21867,
   'OriginStateName': 'Colorado',
   'OriginCityName': 'Denver, CO'}],
 1997: [{'airport': 'ORD',
   'count': 140344,
   'OriginStateName': 'Illinois',
   'OriginCityName': 'Chicago, IL'},
  {'airport': 'DFW',
   'count': 119643,
   'OriginStateName': 'Texas',
   'OriginCityName': 'Dallas/Fort Worth, TX'},
  {'airport': 'STL',
   'count': 106265,
   'OriginStateName': 'Missouri',
   'OriginCityName': 'St. Louis, MO'}],
 2007: [{'airport': 'ATL',
   'count': 200954,
   'OriginStateName': 'Georgia',
   'OriginCityName': 'Atlanta, GA'},
  {'airport': 'ORD',
   'count': 175202,
   'OriginStateName': 'Illinois',
   'OriginCityName': 'Chicago, IL'},
  {'airport': 'DFW',
   'count': 152544,
   'OriginStateName': 'Texas',
   'Origi

24/03/19 14:33:53 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 313008 ms exceeds timeout 120000 ms
24/03/19 14:33:53 WARN SparkContext: Killing executors is not supported by current scheduler.
24/03/19 14:33:58 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$